In [1]:
import cobra
import hashlib
import libsbml
import os

from libsbml import CVTerm

In [2]:
class Subsystem(object):
    def __init__(self):
        self.name = None
        self.id = None
        self.metabolites = list()
        self.reactions = list()
    
    def add_metabolite(self, metabolite):
        if metabolite not in self.metabolites:
            self.metabolites.append(metabolite)
        
    def add_reaction(self, reaction):
        self.reactions.append(reaction)
    
    def reduce_model(self, model):
        # metabolic species
        remove_species = [species.id for species in model.getListOfSpecies() if species.id not in self.metabolites]
        for s_id in remove_species:
            model.removeSpecies(s_id)
        # reactions
        remove_reactions = [reaction.id for reaction in model.getListOfReactions() if reaction.id not in self.reactions]
        for r_id in remove_reactions:
            model.removeReaction(r_id)
        # subsystems
        groups = model.getPlugin("groups")
        for group in groups_plugin.getListOfGroups():
            if group.id != self.id:
                groups.removeGroup(group.id)        
        return model.getSBMLDocument()
        
    def to_json_model(self, base_name, orig_model):
        name = base_name + "_" +  self.name
        doc = orig_model.getSBMLDocument().clone()  
        model = self.reduce_model(doc.getModel())
        
        # convert to json
        libsbml.writeSBMLToFile(doc, out_dir_and_path(name, "xml"))
        model = cobra.io.read_sbml_model(out_dir_and_path(name, "xml"))
        cobra.io.save_json_model(model, out_dir_and_path(name, "json"))
        
def out_dir_and_path(name, dtype):
        out_dir = os.path.join(os.getcwd(), dtype)
        out_path = f"{name}.{dtype}"
        
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        
        return f"{os.path.join(out_dir, out_path)}"

In [3]:
location = "iPAE1146.xml"
doc = libsbml.readSBMLFromFile(location)
model = doc.getModel()

# convert but ignore errors (constant=false for kineticLaw parameters and differently scaled units)
doc.setLevelAndVersion(3,1, False)  
print(f"Converted model to level {doc.getLevel()}, version {doc.getVersion()}")

# enable and get groups plugin
if not doc.isPackageEnabled("groups"):
    doc.enablePackage("http://www.sbml.org/sbml/level3/version1/groups/version1", "groups", True)
groups_plugin = model.getPlugin("groups")

# enable fbc plugin
# if not doc.isPackageEnabled("fbc"):
#    doc.enablePackage("http://www.sbml.org/sbml/level3/version1/fbc/version2", "fbc", True)
# fbc_plugin = model.getPlugin("fbc")
# fbc_plugin.setStrict(False)

# get charge and formula from nodes, add as fbc-attribute
# for species in model.getListOfSpecies():
#    fbc_species_plugin = species.getPlugin("fbc")
#    notes = species.getNotesString()
#    splits = notes.split("<p>")
#    formula = splits[1][9:].strip()[:-4].strip()
#    charge = splits[2][8:].split("</p>")[0].strip()
#    if formula:
#        fbc_species_plugin.setChemicalFormula(formula)
#    if charge and charge != 'NaN':
#        fbc_species_plugin.setCharge(int(charge))

reaction_notes = dict()
for reaction in model.getListOfReactions():
    # fbc_reaction_plugin = reaction.getPlugin("fbc")
    # reaction.setMetaId(reaction.id)
    # get subsystem and EC number
    notes = reaction.getNotesString()
    splits = notes.split("<p>")
    subsystem_info  = splits[2][10:].strip()[:-4]
    # ec_numbers = splits[3][11:].strip()[:-4]
    
    reaction_notes[reaction.id] = subsystem_info
    
    # biological qualifier
    # cv_term = CVTerm(1)
    # BQB_IS
    # cv_term.setBiologicalQualifierType(0)
    # Add enzyme comission numbers as MIRIAM identifiers
    # if ec_numbers:
    #    if "," in ec_numbers:
    #        for ec_num in ec_numbers.split(","):
    #            cv_term.addResource(f"http://identifiers.org/ec-code/{ec_num}")
    #    else:
    #        cv_term.addResource(f"http://identifiers.org/ec-code/{ec_numbers}")
    # if cv_term.getNumResources() > 0:
    #    reaction.addCVTerm(cv_term)
        
    # transfer flux bound information to fbc
    # fbc_reaction_plugin.setLowerFluxBound("LOWER_BOUND")
    # fbc_reaction_plugin.setUpperFluxBound("UPPER_BOUND")

        
for reaction_id, subsystem in reaction_notes.items():
    # If not part of a subsystem, ignore
    if subsystem == "None" or not subsystem:
        continue
        
    # generate simple id from name string (needs a prefix, else it won't work)
    g_id = "s_" + hashlib.md5(subsystem.encode()).hexdigest()
    
    # find or create group for subsystem
    group = groups_plugin.getGroup(g_id)
    if not group:
        group = groups_plugin.createGroup()
        # part-of relationship
        group.setKind("partonomy")
        group.setName(subsystem)
        group.setId(g_id)
    else:
        assert(group.name == subsystem)
    # add reaction as group member    
    member = group.createMember()
    member.setIdRef(reaction_id)

out_location = f"{location.split('.')[0]}_with_groups.xml"
libsbml.writeSBMLToFile(doc, out_location)

Converted model to level 3, version 1


1

In [4]:
groups = groups_plugin.getListOfGroups()
for idx, group in enumerate(groups):
    print(f"\rProcessing subsystems {idx/len(groups)*100:.2f}%", end="")
    subsystem = Subsystem()
    subsystem.name = group.getName().replace(" ", "_").replace(",","").replace("/","")
    subsystem.id = group.getId()
    for member in group.getListOfMembers():
        subsystem.add_reaction(member.id_ref)
        reaction = model.getReaction(member.id_ref)
        for reactant in reaction.getListOfReactants():
            subsystem.add_metabolite(reactant.species)
        for product in reaction.getListOfProducts():
            subsystem.add_metabolite(product.species)
        for modifier in reaction.getListOfModifiers():
            subsystem.add_metabolite(modifier.species)
    
    subsystem.to_json_model(location.split('.')[0], model)

print("\nDone")

Processing subsystems 98.68%
Done
